<h1>Project D4.5</h1>
<p>Este projeto <i>jupyter notebook</i> é uma forma de mostrar a implementação de cada arquivo para o algoritmo C4.5</p>
<p>Ao contrário do arquivo de definição e exploração este arquivo se concentra em ter somente código python</p>
<p>Cada célula seria um arquivo a estar no mesmo diretório e a indicação do nome deste arquivo esta no comentário junto com uma breve descrição</p>

In [1]:
#####
# Filename: DT_DataStructure.py
# Description: Contém a definição dos tipos de dados utilizados no projeto
###
from typing import List, Dict, Any

# Alias for type
DataFrequency = Dict[str, Any]

Frequency = List[DataFrequency]

DataIdColumn = List[str]

class Struct:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)
    
class StructNode(Struct):
    pass

In [37]:
#####
# Filename: DT_Util.py
# Description: Contém funções utilizadas pelas classes, mas que não fazem parte do comportamentpo das classes
###

# import DT_DataStructure.py

class Util:
    """Class Util"""

In [52]:
#####
# Filename: DT_DecisionTree.py
# Description: Contém a classe para arvore de decisão
###

# import DT_DataStructure

class DecisionTree:
    """Decision Tree Class"""
    def __init__(self, data_trainning, column_label_class: str, target_class: str):
        from pandas import DataFrame
        
        if not isinstance(data_trainning, DataFrame):
            raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
        self.__data_trainning = data_trainning.iloc[:,0:]
        
        self.__total_instances = self.__data_trainning.shape[0]
        
        #if (id_column_class >= data_trainning.shape[1]):
        #    raise TypeError('Expected value integer in limit for data frame')
        #self.__id_column_class = id_column_class
        
        self.__entropy_global = 0.0
        
        self.__root_tree = None
        
        self.__logger = ''
        
        self.__target_class = target_class
        
        self.__column_label_class = column_label_class
        
        self.__columns = None
        
        # self.__type_discret = (str)
        
        # self.__type_continuous = ('int', 'float')
        
        self.__method_continuos_supported = ('media', 'mediana', 'quantil', 'moda')
        
        self.__method_continuos_selected = 'media'
        
        self.__method_continuos_args = {'quantil': 0.5}

        
    def __entropy(self, class_positive: int, class_negative: int) -> float:
        from math import log2
        """
            Esta função calcula a entropia total de um conjunto de dados, observe 
                que o valor da entropia varia em função da precisão de ponto flutuante de python
            Input:  class_positive = numero de instâncias com classe positiva
                    class_negative = número de instâncias com classe negativa
            Output: (float) que representa a entropia total do conjunto
        """
        if (class_positive < 0) or (class_negative < 0):
            return -1
        if (class_positive == 0) or (class_negative == 0):
            return 0
        total_instance = class_positive + class_negative
        p = (class_positive/total_instance) * log2(class_positive/total_instance)
        q = (class_negative/total_instance) * log2(class_negative/total_instance)
        r__entropy = (-1 * p) - q
        
        return r__entropy
    
    
    def __prepareCalcEntropy(self, ref_trainning: list) -> float:
        """
             Esta função lida com os dados do subconjunto para o cálculo da entropia, desta maneira 
                 de um lado temos a função que calcula a entropia e de outro uma função que prepara
                 os dados para este cálculo
            Input:  ref_trainning = uma lista com os nomes das colunas (subconjunto) para o cálculo
                        da entropia
            Output: (float) que representa a entropia total do conjunto
        """
        if (not self.__column_label_class in ref_trainning):
            ref_trainning.append(self.__column_label_class)
        
        df_selected = self.__data_trainning.loc[:,ref_trainning]
        total_intances = self.__data_trainning[self.__column_label_class].count()
        mask = self.__column_label_class + ' == ' + '"' + self.__target_class + '"'
        class_positive = df_selected.query(mask)[self.__column_label_class].count()
        class_negative = total_intances - class_positive
        r__prepare_calc_entropy = self.__entropy(class_positive, class_negative)
        
        return r__prepare_calc_entropy
        
        
    def __gaugeStopRecursion(ref_trainning: list) -> bool:
        """ 
            Esta função determina quando a recursividade da arvore deve parar para a expansão
                Neste caso termina caso seja o úlitmo nó da lista
            Input: Dados de treinamento
            Output: (Bool) True = Stop, False = Continue
        """
        if len(ref_trainning) == 1:
            return True
        
        return False
    
    
    def __gainInformation(self, frequencies: Frequency, total_instances: int) -> float:
        """
            Esta função calcula o ganho de informação de um conjunto de dados, observe 
                que o valor do ganho de informação varia em função da precisão de ponto 
                flutuante de python
            Input:  class_positive = numero de instâncias com classe positiva
                    class_negative = número de instâncias com classe negativa
                    frequencies = uma lista de dicionário que representa os dados do conjunto
            Output: (float) que representa o ganho de informação total do conjunto
        """
        gain_local = 0
        
        for frequency in frequencies:        
            frequency_entropy = self.__entropy(frequency['class_positive'], frequency['class_negative'])
            frequency_relative = frequency['class_positive'] + frequency['class_negative']
            gain_local += (frequency_relative / total_instances) * frequency_entropy

        r__gainInformation =  self.__entropy_global - gain_local
        return r__gainInformation
    
    
    def __get_point_cut(self, df_column) -> float:
        """
            Esta função calcula o ponto de divisão em uma série de acordo com as pŕedefinições da classe
            Input:  df_column = Coluna do dataframe
            Output: (float) que representa o valor do ponto de divisão
        """
        r__point_cut = 0
        if (self.__method_continuos_selected == 'media'):
            r__point_cut = df_column.mean()
        elif (self.__method_continuos_selected == 'mediana'):
            r__point_cut = df_column.median()
        elif (self.__method_continuos_selected == 'quantil'):
            r__point_cutter = df_column.quantile(q=self.__method_continuos_args['quantil'])
        elif (self.__method_continuos_selected == 'moda'):
            r__point_cut = df_column.mode()
        else:
            r__point_cut = df_column.mean()
        
        return r__point_cut;
    
    
    def prepareCalcGainInformation(self, partitions: list, df_work ) -> list:
        column_frequency = []
        for partition in partitions:
            mask_up = column + ' == ' + '"' + partition + '"'
            df_uv = df_work.query(mask_uv)
            total_instances = df_uv[column].count()
            mask_target = self.__column_label_class + ' == ' + '"' + self.__target_class + '"'
            class_positive = df_uv.query(mask_target)[column].count()
            class_negative = total_instances - class_positive
            column_frequency.append({'value': unique_value, 
                                     'class_positive': class_positive, 
                                     'class_negative': class_negative
                                    })
            return column_frequency
        
        
    # Deveria haver um parametro column_predecessor para que os dados fossem 
    # filtrados com base no nó anterior cujos valores determinam a classe
    def splitDecisionTree(self, columns_trainning: DataIdColumn) -> None:
        r_split = {}
        
        print(self.__entropy_global)
        
        for column in columns_trainning:
            type_column = type(self.__data_trainning[column][0])
            df_work = self.__data_trainning.loc[:,[self.__column_label_class, column]]
            total_instances_work = df_work[column].count()
            if ( type_column == str):
                column_unique_value = list(df_work[column].unique())
                column_frequency = []
                for unique_value in column_unique_value:
                    mask_uv = column + ' == ' + '"' + unique_value + '"'
                    df_uv = df_work.query(mask_uv)
                    total_instances = df_uv[column].count()
                    mask_target = self.__column_label_class + ' == ' + '"' + self.__target_class + '"'
                    class_positive = df_uv.query(mask_target)[column].count()
                    class_negative = total_instances - class_positive
                    column_frequency.append({'value': unique_value, 
                                             'class_positive': class_positive, 
                                             'class_negative': class_negative
                                            })
                r_split[column] = self.__gainInformation(column_frequency, total_instances_work)
            else:
                point_cut = self.__get_point_cut(df_work[column])
                column_frequency = []
                for unique_partition in ['<', '>=']:
                    mask_up = column + unique_partition + str(point_cut)
                    df_up = df_work.query(mask_up)
                    total_instances = df_up[column].count()
                    mask_target = self.__column_label_class + ' == ' + '"' + self.__target_class + '"'
                    class_positive = df_up.query(mask_target)[column].count()
                    class_negative = total_instances - class_positive
                    column_frequency.append({'value': unique_partition, 
                                             'class_positive': class_positive, 
                                             'class_negative': class_negative
                                            })
                    
                r_split[column] = self.__gainInformation(column_frequency, total_instances_work) 
                    
        return r_split
    
    
    # @ TODO: Reve a criação do nó para o caso de parada de recursão
    def __gender(self, ref_trainning: list) -> Any:
        """
            Esta função gera recursivamente a arvore de decisão
            Input: Dados de treinamento
            Output: (Struct) a arvore de decisão
        """
        if (self.__gaugeStopRecursion(ref_trainning)):
            r__label = ref_trainning[0]
            r__classification = self.identifyClass(ref_trainning)
            r__test_condition = self.trunkCondition(ref_trainning)
            r__leaf_end = StructNode(label = r__label, 
                                     test_condition = r__test_condition, 
                                     leaf = True, 
                                     classification = r__classification)
            return r__leaf_end
        # else
        r__attribPointDivision = self.__splitDecisionTree(ref_trainning)
        
        
    def build(self) -> None:
        """ Esta função inicia o processo para criar a arvore de decisão"""
        column_df = list(self.__data_trainning.columns)
        self.__columns = column_df
        self.__entropy_global = self.__prepareCalcEntropy(column_df)
        
        #column_df.pop(self.__id_column_class)
        #self.__root_tree = self.__gender(column_df)
        
        
    def __gender_old(ref_trainning: List):
        """
            Esta função gera recursivamente a arvore de decisão
            Input: Dados de treinamento
            Output: (Struct) a arvore de decisão
        """
        if gaugeStopRecursion(trainningData):
            atributo = getAttribFromDataTrainning(trainningData)
            classification = identifyClass(trainningData)
            tc = trunkCondition(trainningData)
            leafEnd = StructNode(label = atributo, test_condition = tc, leaf = True, classification = '')    
            return leafEnd
        # else
        attribPointDivision = splitDecisionTree(trainningData)
        tc = trunkCondition(trainningData)
        left_partition = copyDataFrame(trainningData, 0, attribPointDivision)
        right_partition = copyDataFrame(trainningData, attribPointDivision, -1)
        leafRoot = StructNode(label = attribPointDivision, test_condition = tc, leaf = False, left = NULL, right = NULL)

        leafRoot.left = genderDecisionTree(left_partition)
        leafRoot.right = genderDecisionTree(right_partition)
        return leafRoot

    
    def setProperties(propertie: str, value: Any) -> None:
        pass
    
    
    def getProperties(propertie: str) -> str:
        pass

    
    def getAttribFromDataTrainning(trainningData):
        """Extrai o rótulo de um atributo dos dados de treinamento"""
        pass


    def identifyClass(self, trainningData):
        """Identifica qual a classe para determinado atributo"""
        pass

    def trunkCondition(self, trainningData):
        """Retorna qual a regra a ser aplicada em determinado tronco da arvore"""
        pass

    def copyDataFrame(trainningData, begin, end):
        """Retorna uma parte do conjunto de dados mapeando em colunas e linhas"""
        pass
    
    
    


NameError: name 'Float' is not defined

<p>Abaixo esta a seção de testes utilizada</p>

In [5]:
import pandas as pd
df_bc = pd.read_csv('breast-cancer.data')
labels = ['classe', 'age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irradiat']
df_bc = pd.read_csv('breast-cancer.data', names=labels)
df_bc.to_csv('breast-cancer-edited', index=False)

In [37]:
a = DecisionTree(df_bc, 'classe', 'no-recurrence-events')
a.build()
# ['classe', 'age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irradiat']
columns = ['age', 'menopause', 'inv_nodes', 'deg_malig']
a.splitDecisionTree(columns)

0.8778446951746506


{'age': 0.010605956535614025,
 'menopause': 0.0020016149737116518,
 'inv_nodes': 0.06899508808988608,
 'deg_malig': None}

In [51]:
a = DecisionTree(df_bc, 'classe', 'no-recurrence-events')
a.build()
# ['classe', 'age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irradiat']
columns = ['age', 'menopause', 'inv_nodes', 'deg_malig']
a.splitDecisionTree(columns)

0.8778446951746506


{'age': 0.010605956535614136,
 'menopause': 0.0020016149737116518,
 'inv_nodes': 0.06899508808988608,
 'deg_malig': 0.0754168448376169}